In [515]:
import pyomo.environ as pyo
from pyomo.dae import ContinuousSet, DerivativeVar
import numpy as np
import pandas as pd

In [516]:
# Define model
model = pyo.ConcreteModel()

In [517]:
# Global parameters
model.R = pyo.Param(initialize=0.082057) # L·atm/(mol·K)

In [518]:
# Reator Configuration
L = 50  # cm
ID = 0.052  # cm
nx = 21
A = np.pi * ID**2 / 4

model.L = pyo.Param(initialize=L)
model.ID = pyo.Param(initialize=ID)
model.nx = pyo.Param(initialize=nx)
model.A = pyo.Param(initialize=A)

In [519]:
# Domain
model.x = ContinuousSet(bounds=(0, L), initialize=np.linspace(0, L, nx))

In [520]:
# Component properties
comp = ['CO2', 'H2', 'CH4', 'CO', 'H2O']
cp = {'CO2': 37.1, 'H2': 28.8, 'CH4': 35.3, 'CO': 29.1, 'H2O': 33.6}

In [521]:
# Feed conditions
R = 0.082057  # L·atm/(mol·K)
P = 1  # atm
T0 = 273.15  # K
Fvi = 100  # cm3/min
Fi = P*Fvi/1000/60/R/T0  # mol/s
Ti = 473.15  # K
zi_values = {'CO2': 0.2, 'H2':0.8, 'CH4':0.0, 'CO':0.0, 'H2O':0.0}
rhoi = P/R/Ti  # mol/L
Fv = Fi/rhoi  # L/s
V0 = Fv*1000/A  # cm/s

model.P = pyo.Param(initialize=P)
model.T0 = pyo.Param(initialize=T0)
model.Fvi = pyo.Param(initialize=Fvi)
model.Fi = pyo.Param(initialize=Fi)
model.Ti = pyo.Param(initialize=Ti)
model.zi = pyo.Param(comp, initialize=zi_values)
model.rhoi = pyo.Param(initialize=rhoi)
model.Fv = pyo.Param(initialize=Fv)
model.V0 = pyo.Param(initialize=V0)
model.Ci = pyo.Param(comp, initialize={c: rhoi*zi_values[c] for c in comp})
model.cp = pyo.Param(comp, initialize=cp)

In [522]:
# Inputs
Q = 10000 # W

# Thermodynamics

# Initial Conditions
def C_init(model, x, c):
    return model.Ci[c]

def z_init(model, x, c):
    return model.zi[c]

def T_init(model, x):
    return model.Ti

def cpx_init(model, x):
    total = 0
    for ci in comp:
        total += model.cp[ci] * model.zi[ci]
    return total

model.C = pyo.Var(model.x, comp, domain=pyo.NonNegativeReals, initialize=C_init)
model.dCdx = DerivativeVar(model.C, wrt=model.x)
model.rho = pyo.Var(model.x, domain=pyo.NonNegativeReals, initialize=model.rhoi)
model.drhodx = DerivativeVar(model.rho, wrt=model.x)
model.z = pyo.Var(model.x, comp, domain=pyo.NonNegativeReals, initialize=z_init)
model.V = pyo.Var(model.x, domain=pyo.NonNegativeReals, initialize=model.V0)
model.dVdx = DerivativeVar(model.V, wrt=model.x)
model.T = pyo.Var(model.x, domain=pyo.NonNegativeReals, initialize=T_init)
model.dTdx = DerivativeVar(model.T, wrt=model.x)
model.cpx = pyo.Var(model.x, domain=pyo.NonNegativeReals, initialize=cpx_init)

# Algebraic Equations
def cal_rho(model, x):
    return model.rho[x] == model.P / (model.R * model.T[x])
model.rho_constr = pyo.Constraint(model.x, rule=cal_rho)

def cal_z(model, x, c):
    return model.z[x, c] == model.C[x, c] / model.rho[x]
model.z_constr = pyo.Constraint(model.x, comp, rule=cal_z)

def cal_cpx(model, x):
    return model.cpx[x] == sum(model.cp[c] * model.z[x, c] for c in comp)
model.cpx_constr = pyo.Constraint(model.x, rule=cal_cpx)

# Balance Equations
def continuity(model, x):
    return model.rho[x] * model.dVdx[x] + model.drhodx[x] * model.V[x] == 0
model.continuity_constr = pyo.Constraint(model.x, rule=continuity)

def component_balance(model, x, c):
    return model.V[x] * model.dCdx[x, c] + model.C[x, c] * model.dVdx[x] == 0
model.component_balance_constr = pyo.Constraint(model.x, comp, rule=component_balance)

def energy_balance(model, x):
    return model.rho[x] * model.V[x] * model.cpx[x] * model.dTdx[x] == Q
model.energy_balance_constr = pyo.Constraint(model.x, rule=energy_balance)

# Boundary Conditions
def init_C(model, comp):
    return model.C[0, comp] == model.Ci[comp]
model.init_C_constr = pyo.Constraint(comp, rule=init_C)

def init_V(model):
    return model.V[0] == model.V0
model.init_V_constr = pyo.Constraint(rule=init_V)

def init_T(model):
    return model.T[0] == model.Ti
model.init_T_constr = pyo.Constraint(rule=init_T)

In [523]:
# Apply finite difference discretization
discretizer = pyo.TransformationFactory('dae.finite_difference')
discretizer.apply_to(model, nfe=nx-1, wrt=model.x, scheme='BACKWARD')

In [524]:
solver = pyo.SolverFactory('ipopt')
result = solver.solve(model, tee=True)

Ipopt 3.14.19: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:     1558
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      525

Total number of variables............................:      462
                     variables with only lower bounds:      294
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      461
Total number

In [525]:
# 모든 x 축에 대해 V, rho, C, z 값을 출력하고, 체계적으로 정렬하여 저장
import re
profile_data = []

# V 값 저장
for i in model.x:
    profile_data.append(['V', i, '', model.V[i].value])

# rho 값 저장
for i in model.x:
    profile_data.append(['rho', i, '', model.rho[i].value])

# T 값 저장
for i in model.x:
    profile_data.append(['T', i, '', model.T[i].value])

# cpx 값 저장
for i in model.x:
    profile_data.append(['cpx', i, '', model.cpx[i].value])

# C[x, c] 값 저장
for c in comp:
    for i in model.x:
        profile_data.append(['C', i, c, model.C[i, c].value])

# z[x, c] 값 저장
for c in comp:
    for i in model.x:
        profile_data.append(['z', i, c, model.z[i, c].value])

# 정렬: 변수명 -> 성분명 -> x좌표 순 (같은 성분명끼리 묶임)
def sort_key(item):
    var_name, x_coord, component, value = item
    # 변수명 -> 성분명 -> x좌표 순으로 정렬
    return (var_name, component, float(x_coord))

sorted_data = sorted(profile_data, key=sort_key)

# DataFrame 생성
df_profile = pd.DataFrame(sorted_data, columns=['변수명', 'x좌표', '성분명', '값'])

# CSV 저장
df_profile.to_csv('profile.csv', index=False)

print("Profile saved to profile.csv (grouped by component)")
print(df_profile.head(20))

Profile saved to profile.csv (grouped by component)
   변수명   x좌표  성분명             값
0    C   0.0  CH4  0.000000e+00
1    C   2.5  CH4  9.366615e-41
2    C   5.0  CH4  8.946289e-41
3    C   7.5  CH4  8.562753e-41
4    C  10.0  CH4  8.211419e-41
5    C  12.5  CH4  7.888266e-41
6    C  15.0  CH4  7.590008e-41
7    C  17.5  CH4  7.313869e-41
8    C  20.0  CH4  7.057446e-41
9    C  22.5  CH4  6.818696e-41
10   C  25.0  CH4  6.595798e-41
11   C  27.5  CH4  6.387239e-41
12   C  30.0  CH4  6.191642e-41
13   C  32.5  CH4  6.007848e-41
14   C  35.0  CH4  5.834801e-41
15   C  37.5  CH4  5.671567e-41
16   C  40.0  CH4  5.517328e-41
17   C  42.5  CH4  5.371370e-41
18   C  45.0  CH4  5.245196e-41
19   C  47.5  CH4  5.114398e-41
